<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/workflow_series_01_building_a_multi_round_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's Get Started by Building a Multi-Round Chat with Agently Workflow! `#Agently_Workflow_Showcase_Series - 01`

## Demo Description

**Author:** Agently Team

**Description:**

We're glad to introduce a brand new feature of **_<font color = "red">Agent</font><font color = "blue">ly</font>_** v3.2 to you all: **`Agently Workflow`**!

With this new feature, you can arrange and manage your LLMs based application workflow **in just 3 steps, simple and easy:**

1. Define and program your application logic into different `workflow chunks`;
2. Connect chunks using `chunk.connect_to()` in orders; (Loop and condition judugment supported)
3. Start the workflow using `workflow.startup()`.

This showcase is the first one of the **`Agently Workflow Showcase Series`**, more showcases will coming soon.

In this case, we will show you how to use **`Agently Workflow`** to create a multi-round chat workflow:

我们非常高兴能够向您介绍**_<font color = "red">Agent</font><font color = "blue">ly</font>_** v3.2版本推出的全新功能：**`Agently Workflow`**！

使用这项全新的功能，只需要一二三步走，您就能够轻松惬意地编排管理您的语言模型应用工作流了：

1. 在`workflow chunks`切块中编写您的应用工作流中的单块工作逻辑（如输入、判断、请求执行、数据存取等）；
2. 使用`chunk.connect_to()`方法将切块按您想要的工作顺序进行连接（支持环状连接、条件分支等复杂连接关系）；
3. 通过`workflow.startup()`开始运行工作流。

本案例是**`Agently Workflow系列案例`**的第一个案例，后续我们还将更新更多相关案例。

在本案例中，我们将向您演示如何使用**Agently Workflow**功能创建一个多轮对话的工作流：

## Step 1: Install Packages

In [ ]:
!pip install -U Agently>=3.2

> ℹ️ Agently Workflow is a new feature in version >= `3.2.0`

## Step 2: Create a Multi-Round Chat Workflow in the Basic Way

In [ ]:
import Agently
import logging

agent_factory = (
        Agently.AgentFactory()
        .set_settings("model.Google.auth.api_key", "")
        .set_settings("current_model", "Google")
)

# Step 0. Create Workflow Instance and Agent Instance
workflow = Agently.Workflow()
workflow.executor.settings.update({ "logger_level": logging.WARNING })
agent = agent_factory.create_agent()

# Step 1. Create Chunks
## Start Chunk
start_chunk = workflow.schema.create_chunk(
    title = "Multi-Round Chat",
    type = "Start"
)

## User Input Chunk
user_input_chunk = workflow.schema.create_chunk(
    title = "User Input",
    handles = {
        "inputs": [{ "handle": "nothing" }],
        "outputs": [{ "handle": "user_input" }],
    },
    executor = lambda inputs_pkg, store: {
        "user_input": input("[User]: ")#<- You can change it to an API/WS request
    },# You can use a lambda function as an executor
)

## Assistant Reply Chunk
# You can also define an normal executor function to handle more complex logic
# get data from input handles by `input_pkg["<handle name>"]`
# get data from cache storage by `store.get("<key>")`
# save data to cache storage by `store.save("<key>", <value>)`
def assistant_reply_executor(inputs_pkg, store):
    chat_history = store.get("chat_history") or []
    reply = (
        agent
            .chat_history(chat_history)
            .input(inputs_pkg["user_input"])
            .start()
    )
    print("[Assistant]: ", reply)
    chat_history.append({ "role": "user", "content": inputs_pkg["user_input"] })
    chat_history.append({ "role": "assistant", "content": reply })
    store.save("chat_history", chat_history)
    return
assistant_reply_chunk = workflow.schema.create_chunk(
    title = "Assistant Reply",
    handles = {
        "inputs": [{ "handle": "user_input" }],
        "outputs": [{ "handle": "nothing" }],
    },
    executor = assistant_reply_executor,
)

# Step 2. Connect Chunks in Orders
start_chunk.connect_to(user_input_chunk)
user_input_chunk.connect_to(assistant_reply_chunk)
## Created a loop by connect tail chunk to head chunk
assistant_reply_chunk.connect_to(user_input_chunk)

# Step 3. Startup Workflow
workflow.startup()

[User]: Hello
[Assistant]:  Hello there! How can I assist you today?
[User]: Tell me one tip about Python programing
[Assistant]:  **Tip:** Use a debugger to step through your code line by line and identify errors or unexpected behavior. This can save a lot of time compared to printing statements or using logging to debug.
[User]: Tell me one tip about Node.js programing
[Assistant]:  **Tip:** Leverage the Node.js event loop to handle asynchronous operations efficiently. The event loop allows Node.js to perform non-blocking I/O operations, making it well-suited for building scalable and responsive applications.
[User]: Tell me one tip about using Gemini Pro
[Assistant]:  **Tip:** Utilize the Gemini Pro API to automate your trading strategies and access real-time market data. The API provides a comprehensive set of endpoints that allow you to interact with the exchange, place orders, manage positions, and retrieve market information.
[User]: What did you just talk about?
[Assistant]:  I

---

[**_<font color = "red">Agent</font><font color = "blue">ly</font>_** Framework - Speed up your AI Agent Native application development](https://github.com/Maplemx/Agently)